In [5]:
from irace2 import irace, dummy_stats_test
from sampling_functions import norm_sample, truncated_poisson, truncated_skellam
import numpy as np
import random
import scipy.stats as stats
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tqdm import tqdm
from sklearn import preprocessing
import pandas as pd
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.stats.weightstats as stats
import scipy.stats as ss
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,cross_val_score
from scipy.stats import norm, poisson, skellam
from copy import copy, deepcopy

## Get Data

In [6]:
df = pd.read_csv('spect_train.csv')
X = preprocessing.normalize(df.drop(columns=['OVERALL_DIAGNOSIS']).to_numpy())
y = df['OVERALL_DIAGNOSIS'].to_numpy()

## Models

In [7]:
#all the parameters being configures must be set beforehand
models = [LogisticRegression(C=1), 
    RandomForestClassifier(n_estimators=100,max_depth=5,ccp_alpha=0.0),
    SVC(C=1,coef0=0.0),
    XGBClassifier(n_estimators=100,max_depth=6,subsample=1)]


parameters_dict = {
    'LogisticRegression': {'C': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
                            'penalty':['l2'],
                            'solver':['lbfgs','newton-cg','sag']},
    'SVC':{'C':lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'coef0': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'kernel':['linear','poly','rbf','sigmoid'],
            'decision_function_shape':['ovo','ovr']},
    'RandomForestClassifier': {'n_estimators': lambda loc: truncated_skellam(loc, mu1=10, mu2=10, min=1), 
                                'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                                'max_features':['sqrt', 'log2', None],
                                'ccp_alpha':lambda loc : norm_sample(loc=loc, scale=0.1, min= 1e-3)
                                },
    'XGBClassifier': {'sample_type': ['uniform','weighted'], 
                        'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                        'booster':['gbtree','dart'],
                        'subsample':lambda loc : norm_sample(loc=loc, scale=0.3, min= 1e-2,max=1)}
}

## Run race

In [4]:
stat_tests = [ ss.ttest_rel,
                ss.ttest_ind,
                ss.mannwhitneyu,
                ss.wilcoxon,
                dummy_stats_test] 

stat_test = ss.mannwhitneyu #stats.ttest_ind, stats.mannwhitneyu

best_model,best_scores,pop, pop_scores = irace(models, X, y, lambda x: x > 500, stat_test, parameters_dict, pop_size = 30, scoring='f1_macro', r=30)

Gen 0

Average scores: 0.7005761309264343
Gen 1

Average scores: 0.7342370903143949
Gen 2

Average scores: 0.7406184277654254
Gen 3

Average scores: 0.742060751025905
Gen 4

Average scores: 0.7459508314503202
Gen 5

Average scores: 0.7459508314503202
Gen 6

Average scores: 0.7473225588339446
Gen 7

Average scores: 0.7473225588339446
Gen 8

Average scores: 0.7473225588339446
Gen 9

Average scores: 0.7473225588339446
Gen 10

Average scores: 0.7502946157530602
Gen 11

Average scores: 0.7524366904870787
Gen 12

Average scores: 0.7524366904870787
Gen 13

Average scores: 0.7524366904870787
Gen 14

Average scores: 0.7524366904870787
Gen 15

Average scores: 0.7524366904870787
Gen 16

Average scores: 0.7524366904870787
Gen 17

Average scores: 0.7524366904870787
Gen 18

Average scores: 0.7552156244561916
Gen 19

Average scores: 0.7552156244561916
Gen 20

Average scores: 0.7552156244561916
Gen 21

Average scores: 0.7552156244561916
Gen 22

Average scores: 0.7552156244561916
Gen 23

Average scores

c:\Users\rcpsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Average scores: 0.7702946848746642
Gen 99

Average scores: 0.7702946848746642
Gen 100

Average scores: 0.7702946848746642
Gen 101

Average scores: 0.7702946848746642
Gen 102

Average scores: 0.7702946848746642
Gen 103

Average scores: 0.7702946848746642
Gen 104

Average scores: 0.7702946848746642
Gen 105

Average scores: 0.7702946848746642
Gen 106

Average scores: 0.7702946848746642
Gen 107

Average scores: 0.7702946848746642
Gen 108

Average scores: 0.7702946848746642
Gen 109

Average scores: 0.7702946848746642
Gen 110

Average scores: 0.7702946848746642
Gen 111

Average scores: 0.7702946848746642
Gen 112

Average scores: 0.7702946848746642
Gen 113

Average scores: 0.7702946848746642
Gen 114

Average scores: 0.7702946848746642
Gen 115

Average scores: 0.7702946848746642
Gen 116

Average scores: 0.7702946848746642
Gen 117

Average scores: 0.7702946848746642
Gen 118

Average scores: 0.7702946848746642
Gen 119

Average scores: 0.7702946848746642
Gen 120

Average scores: 0.770294684874664

c:\Users\rcpsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Average scores: 0.7723658571485782
Gen 129

Average scores: 0.7723658571485782
Gen 130

Average scores: 0.7723658571485782
Gen 131

Average scores: 0.7723658571485782
Gen 132

Average scores: 0.7723658571485782
Gen 133

Average scores: 0.7723658571485782
Gen 134

Average scores: 0.7723658571485782
Gen 135

Average scores: 0.7723658571485782
Gen 136

Average scores: 0.7723658571485782
Gen 137

Average scores: 0.7723658571485782
Gen 138

Average scores: 0.7723658571485782
Gen 139

Average scores: 0.7723658571485782
Gen 140

Average scores: 0.7749264633088769
Gen 141

Average scores: 0.7749264633088769
Gen 142

Average scores: 0.7749264633088769
Gen 143

Average scores: 0.7749264633088769
Gen 144

Average scores: 0.7749264633088769
Gen 145

Average scores: 0.7749264633088769
Gen 146

Average scores: 0.7749264633088769
Gen 147

Average scores: 0.7749264633088769
Gen 148

Average scores: 0.7749264633088769
Gen 149

Average scores: 0.7749264633088769
Gen 150

Average scores: 0.77492646330887

...

c:\Users\rcpsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Average scores: 0.7786821387027495
Gen 265

Average scores: 0.7786821387027495
Gen 266

Average scores: 0.7786821387027495
Gen 267

Average scores: 0.7786821387027495
Gen 268

Average scores: 0.7786821387027495
Gen 269

Average scores: 0.7786821387027495
Gen 270

Average scores: 0.7786821387027495
Gen 271

Average scores: 0.7786821387027495
Gen 272

Average scores: 0.7786821387027495
Gen 273

Average scores: 0.7786821387027495
Gen 274

Average scores: 0.7786821387027495
Gen 275

Average scores: 0.7786821387027495
Gen 276

Average scores: 0.7786821387027495
Gen 277

Average scores: 0.7786821387027495
Gen 278

Average scores: 0.7786821387027495
Gen 279

Average scores: 0.7786821387027495
Gen 280

Average scores: 0.7786821387027495
Gen 281

Average scores: 0.7786821387027495
Gen 282

Average scores: 0.7786821387027495
Gen 283

Average scores: 0.7786821387027495
Gen 284

Average scores: 0.7786821387027495
Gen 285

Average scores: 0.7786821387027495
Gen 286

Average scores: 0.77868213870274

c:\Users\rcpsi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Average scores: 0.7786821387027495
Gen 410

Average scores: 0.7786821387027495
Gen 411

Average scores: 0.7786821387027495
Gen 412

Average scores: 0.7786821387027495
Gen 413

Average scores: 0.7786821387027495
Gen 414

Average scores: 0.7786821387027495
Gen 415

Average scores: 0.7786821387027495
Gen 416

Average scores: 0.7786821387027495
Gen 417

Average scores: 0.7786821387027495
Gen 418

Average scores: 0.7786821387027495
Gen 419

Average scores: 0.7786821387027495
Gen 420

Average scores: 0.7786821387027495
Gen 421

Average scores: 0.7786821387027495
Gen 422

Average scores: 0.7786821387027495
Gen 423

Average scores: 0.7786821387027495
Gen 424

Average scores: 0.7786821387027495
Gen 425

Average scores: 0.7786821387027495
Gen 426

Average scores: 0.7786821387027495
Gen 427

Average scores: 0.7786821387027495
Gen 428

Average scores: 0.7786821387027495
Gen 429

Average scores: 0.7786821387027495
Gen 430

Average scores: 0.7786821387027495
Gen 431

Average scores: 0.77868213870274

KeyboardInterrupt: 

In [ ]:
print()
for i in range(len(pop)):
    print(pop[i])
    scores = cross_val_score(pop[i], X, y, cv=10, scoring='f1') 
    print(f'{np.mean(scores)} +- {np.std(scores)}')

In [ ]:
print()
for i in range(len(pop)):
    print(pop[i]) 
    print(f'{np.mean(pop_scores[i])} +- {np.std(pop_scores[i])}')

In [ ]:
print('=======================================')
print(best_model)
print(f'{np.mean(best_scores)}+-{np.std(best_scores)}')

In [ ]:
scores = cross_val_score(SVC(), X, y, cv=10, scoring='f1')
print(f'{np.mean(scores)} +- {np.std(scores)}')